In [4]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [10]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView("home_sales")


In [11]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query_1="""
SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year(date)
ORDER BY year
"""
avg_price_four_bed = spark.sql(query_1)
avg_price_four_bed.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT year(date_built) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY year(date_built)
ORDER BY year
"""
avg_price_three_bed_bath = spark.sql(query_2)
avg_price_three_bed_bath.show()


AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "year(date_built)" due to data type mismatch: Parameter 1 requires the "DATE" type, however "date_built" has the type "INT".; line 2 pos 7;
'Sort ['year ASC NULLS FIRST], true
+- 'Aggregate [year(date_built#19)], [year(date_built#19) AS year#74, round(avg(price#20), 2) AS avg_price#75]
   +- Filter ((bedrooms#21 = 3) AND (bathrooms#22 = 3))
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27])
            +- Relation [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27] csv


In [27]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY year
"""
avg_price_three_bed_bath = spark.sql(query_2)
avg_price_three_bed_bath.show()

+----+---------+
|year|avg_price|
+----+---------+
|2010|292859.62|
|2011|291117.47|
|2012|293683.19|
|2013|295962.27|
|2014|290852.27|
|2015| 288770.3|
|2016|290555.07|
|2017|292676.79|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query_2 = """
SELECT date_built as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY year
"""
avg_price_three_bed_bath = spark.sql(query_2)
avg_price_three_bed_bath.show()

In [29]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3 = """
SELECT year(date_built) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY year(date_built)
ORDER BY year
"""
avg_price_specific_conditions = spark.sql(query_3)
avg_price_specific_conditions.show()

AnalysisException: [DATATYPE_MISMATCH.UNEXPECTED_INPUT_TYPE] Cannot resolve "year(date_built)" due to data type mismatch: Parameter 1 requires the "DATE" type, however "date_built" has the type "INT".; line 2 pos 7;
'Sort ['year ASC NULLS FIRST], true
+- 'Aggregate [year(date_built#19)], [year(date_built#19) AS year#459, round(avg(price#20), 2) AS avg_price#460]
   +- Filter (((bedrooms#21 = 3) AND (bathrooms#22 = 3)) AND ((floors#25 = 2) AND (sqft_living#23 >= 2000)))
      +- SubqueryAlias home_sales
         +- View (`home_sales`, [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27])
            +- Relation [id#17,date#18,date_built#19,price#20,bedrooms#21,bathrooms#22,sqft_living#23,sqft_lot#24,floors#25,waterfront#26,view#27] csv


In [30]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query_3 = """
SELECT date_built as year, ROUND(AVG(price), 2) as avg_price # Removed the year() function as date_built is already an integer representing the year
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY year
"""
avg_price_specific_conditions = spark.sql(query_3)
avg_price_specific_conditions.show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '#'.(line 2, pos 61)

== SQL ==

SELECT date_built as year, ROUND(AVG(price), 2) as avg_price # Removed the year() function as date_built is already an integer representing the year
-------------------------------------------------------------^^^
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY year


In [17]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00013637542724609375 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [20]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 8.153915405273438e-05 seconds ---


In [21]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "/mnt/data/home_sales_partitioned"
df.write.partitionBy("date_built").parquet(output_path)

In [22]:
# 11. Read the parquet formatted data.
partitioned_df = spark.read.parquet(output_path)

In [23]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales_partitioned")

In [24]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 7.390975952148438e-05 seconds ---


In [25]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")


In [26]:
# 15. Check if the home_sales is no longer cached
print(spark.catalog.isCached('home_sales'))


False
